In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

In [4]:
import json
from tqdm import tqdm

In [5]:
input_data = "docs/test_lv2.json"

In [6]:
# description_agent.ipynb 파일의 generate_detailed_prompt 함수 수정

def generate_detailed_prompt(description: str, snippet: str, module: str) -> str:
    """
    LLM에게 전달할 프롬프트를 생성합니다.
    요구사항의 상세 설명을 최대한 자세하고 체계적으로 작성하도록 유도합니다.
    """

    # 입력 값 확인 (비어 있을 경우 기본값 사용)
    description = description if description else "요구사항 명칭 누락"
    snippet = snippet if snippet else "관련 원문 스니펫 정보 없음"
    module = module if module else "담당 모듈 미지정"

    prompt = f"""
당신은 금융 정보 시스템 구축 프로젝트의 수석 시스템 분석가(SA)입니다. 당신의 임무는 주어진 간략한 요구사항 정보를 바탕으로, 개발팀이 즉시 상세 설계 및 개발에 착수할 수 있도록 매우 구체적이고, 체계적이며, 완전한 '상세 설명'을 작성하는 것입니다.

현재 작성해야 할 요구사항의 기본 정보는 다음과 같습니다:
- [요구사항 명칭]: {description}
- [관련 원문 스니펫]: {snippet}
- [예상 담당 모듈]: {module}

**상세 설명 작성 지침:**

아래 지침에 따라 요구사항의 상세 설명을 작성하십시오. 요구사항의 성격(신규 개발, 기존 기능 개선, 단순 기능 유지, 기존 기능 변경 등)은 제공된 '[요구사항 명칭]'과 '[관련 원문 스니펫]'의 내용을 통해 **스스로 추론**하여 가장 적합한 형식으로 작성해야 합니다. 만약 성격 구분이 명확하지 않거나 정보가 부족할 경우, **"1. 신규 또는 개선 요구사항의 경우"** 지침을 기본으로 적용하여 최대한 상세히 작성하십시오.

**1. 신규 또는 개선 요구사항의 경우:**
   다음과 같은 세부 항목을 반드시 포함하여, 각 항목에 대해 가능한 모든 정보를 구체적으로 기술합니다.

   `[요구사항]`
   - (여기에 '{description}'을 바탕으로 핵심 요구사항 내용을 명확하고 간결하게 재기술합니다. 원문 스니펫의 내용을 참조하여 프로젝트의 목표와 부합하도록 구체화합니다.)

   `[대상업무]`
   - 이 요구사항이 실제 적용될 대상 업무 시스템 또는 서비스 영역을 명확히 명시합니다. (예: 인터넷뱅킹 개인고객 서비스, 여신 심사 시스템, 내부 자금 관리 시스템 등)
   - 해당 요구사항을 구현하는 주요 기능 유형(들)을 상세히 명시합니다. (예: 사용자 인터페이스 화면 개발, 신규 온라인 거래 서비스 개발, 일일/월간 배치 프로그램 개발, 외부 기관 연동 API 인터페이스 개발, 데이터베이스 스키마 변경 및 마이그레이션 등)

   `[요건처리 상세]`
   - **1. 구현 방안 및 처리 프로세스/흐름:**
     - 사용자의 행동 흐름(User Story/Use Case 관점) 또는 데이터의 처리 흐름(Data Flow)을 중심으로, 요구사항이 구현될 방식을 단계별로 아주 상세하게 기술합니다.
     - 데이터 입력: 어떤 화면/채널/시스템을 통해 어떤 데이터 항목들이 입력되는지, 각 데이터의 형식, 유효성 검증 규칙 등을 명시합니다.
     - 데이터 처리: 입력된 데이터를 사용하여 어떤 계산, 변환, 비즈니스 로직이 수행되는지, 관련 알고리즘이나 주요 규칙을 구체적으로 설명합니다. 어떤 내부/외부 시스템과 정보를 주고받는지 명시합니다.
     - 데이터 출력/결과: 처리 결과가 사용자 화면에 어떻게 표시되는지(UI/UX 고려), 어떤 데이터가 데이터베이스에 저장/수정/삭제되는지, 어떤 보고서가 생성되는지, 어떤 알림(SMS, 이메일 등)이 발송되는지 등을 상세히 기술합니다.
     - 화면 설계(해당 시): 주요 화면의 구성 요소(입력 필드, 버튼, 그리드, 차트 등), 화면 간의 네비게이션 흐름, 사용자 인터랙션 방식을 구체적으로 기술합니다. (와이어프레임이나 목업 수준으로 상상하여 기술)
     - 예외 처리: 발생 가능한 주요 오류 상황(예: 입력값 오류, 시스템 장애, 연동 실패 등)과 각 상황에 대한 시스템의 대응 방안(오류 메시지, 복구 절차 등)을 명시합니다.
   - **2. 관리적인 내용 (요구사항 기술 이력 및 관련 이슈):**
     - (주어진 정보나 원문 스니펫에서 추론 가능하다면) 이 요구사항이 도출된 주요 배경이나 근거 자료(예: PI 사업 결과, 현행 시스템 분석 결과, 특정 부서 인터뷰, 사업수행계획서의 특정 항목 등)를 간략히 언급합니다.
     - (주어진 정보나 원문 스니펫에서 추론 가능하다면) 해당 요구사항의 주요 기술 이력(예: 최초 제기 일자, 주요 변경 사항, 버전 정보 등)이나 관련된 중요 이슈(예: 타 팀과의 업무 협의 필요 사항, 특정 기술적 제약으로 인한 기능 폐기 후 재요청 건, 법규 변경으로 인한 긴급 추가 사항 등)가 있다면 구체적으로 기술합니다.
     - 정보가 없거나 추론이 불가능한 경우, "신규 개발 건으로 특이 이력/이슈 현재까지 파악되지 않음. (추후 관련 정보 발생 시 업데이트 예정)" 또는 유사한 표현으로 명시합니다.
   - **3. 기타 고려사항 (비기능적 요건 등):**
     - 성능 목표치(예: 특정 거래의 평균 응답 시간 3초 이내, 동시 사용자 1,000명 처리 가능), 보안 요구사항(예: 개인정보 암호화, 특정 구간 SSL/TLS 적용), 데이터 정합성 유지 방안, 사용성, 확장성, 유지보수성 등 비기능적인 측면에서 특별히 고려해야 할 사항이 있다면 구체적으로 기술합니다.

**2. 유지 대상 요구사항의 경우 (원문 스니펫이나 명칭에서 '유지', '현행 유지', 'AS-IS 기능' 등의 명확한 힌트가 있을 때):**
   `[현행 기능 상세 설명]`
   - (주어진 정보를 바탕으로) 현재 시스템에서 해당 기능이 **어떻게 구현되어 운영되고 있는지**를 현행시스템분석서 수준으로 상세히 기술합니다.
   - 주요 기능 명세, 데이터 흐름, 사용자 인터페이스 구성 및 특징 (해당 시), 관련 테이블 및 주요 데이터 항목, 현재 알려진 문제점이나 개선 요구사항(단순 유지이므로 개선은 제외하고 현상만 기술) 등을 포함합니다. (주어진 정보가 부족하면, "제공된 정보 기반으로 추론한 현행 기능은 다음과 같음..." 형식으로 작성)

**3. 변경된 요구사항의 경우 (원문 스니펫이나 명칭에서 '변경', '수정', '개선' 등의 명확한 힌트가 있고, 변경 내용이 구체적일 때):**
   `[요구사항 변경 상세]`
   - **1. 변경 개요:**
     - 요구사항의 어떤 부분이 어떻게 변경되었는지 핵심 내용을 요약합니다.
     - 변경 사유 및 목적을 명확히 기술합니다.
   - **2. AS-IS (변경 전):**
     - 변경 전의 기능 명세, 처리 방식, 문제점 등을 간략히 기술합니다.
   - **3. TO-BE (변경 후):**
     - 변경된 기능 명세, 처리 방식, 개선점 등을 상세히 기술합니다. (신규/개선 요구사항의 `[요건처리 상세]` 형식 참조)
   - **4. 변경으로 인한 영향:**
     - 다른 시스템, 모듈, 데이터, 사용자 인터페이스 등에 미치는 영향을 분석하여 기술합니다.
   - **5. 관리 구분:** 변경
   - **6. 최종 변경일자:** (정보가 있다면 실제 날짜 명시, 없다면 "최근 변경된 것으로 추정됨"으로 기술)

**작성 사례 (신규/개선 요구사항의 예시):**
"""
    return prompt

In [7]:
def get_detailed_description(description, snippet, module):
    prompt = generate_detailed_prompt(description, snippet, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 시스템 분석 전문가이며, 업무 설명을 상세하게 기술하는 역할입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("Error:", e)
        return "상세 설명 생성 실패"

In [8]:
# JSON 불러오기
with open(input_data, "r", encoding="utf-8") as f:
    requirements = json.load(f)

In [9]:
# 상세 설명 생성
for req in tqdm(requirements, desc="🔍 상세 설명 생성 중"):
    desc = req.get("description", "")
    snippet = req.get("raw_text_snippet", "")
    module = req.get("responsible_module", "")
    req["detailed_description"] = get_detailed_description(desc, snippet, module)

# 결과 저장
detailed_output_path = "docs/test_lv3.json"
with open(detailed_output_path, "w", encoding="utf-8") as f:
    json.dump(requirements, f, ensure_ascii=False, indent=4)

print(f"✅ 상세 설명이 생성되어 저장되었습니다: {detailed_output_path}")

🔍 상세 설명 생성 중: 100%|██████████| 6/6 [03:05<00:00, 30.90s/it]

✅ 상세 설명이 생성되어 저장되었습니다: docs/test_lv3.json
